In [2]:
import numpy as np
import pandas as pd
from scipy.signal import welch
import math

In [3]:
def bandpower(data, sf, band, window_sec=None):
    band = np.array(band)
    freqs, psd = welch(data, sf, nperseg=int(window_sec * sf) if window_sec else None)
    idx_band = np.logical_and(freqs >= band[0], freqs <= band[1])
    return np.trapz(psd[idx_band], freqs[idx_band])  # Integral over the band


In [4]:
bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 12),
    'beta': (12, 30),
    'gamma': (30, math.inf) 
}


In [14]:
def extract_spectral_features(eeg_df, sf=250):
    features = []

    for i, row in eeg_df.iterrows():
        signal = row.values  # 1D EEG signal
        person_features = {}
        total_power = bandpower(signal, sf, (0.5, 100))  # Total power for normalization

        for band_name, band_range in bands.items():
            power = bandpower(signal, sf, band_range)
            person_features[f"{band_name}_power"] = power
            person_features[f"{band_name}_rel_power"] = power / total_power if total_power != 0 else 0
        
        features.append(person_features)

    return pd.DataFrame(features)


In [5]:
data = pd.read_csv('../data/preprocessed.csv')
data.describe()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,169,170,171,172,173,174,175,176,177,178
count,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,...,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000
mean,2499.000000,2.164633,-0.554607,-0.564193,-0.540193,-0.649433,-0.753712,-1.075914,-1.181350,-1.451674,...,1.190700,1.485347,1.662399,1.745655,1.896240,2.166390,2.260436,2.391720,2.331582,2.544677
std,1443.231328,1.153444,17.789305,17.597088,17.681466,17.723401,17.403211,17.384649,17.279831,17.366649,...,17.849879,17.741248,17.593795,17.586370,17.837506,17.770479,17.780897,17.603510,17.718653,17.526130
min,0.000000,0.000000,-146.094760,-82.371185,-81.849570,-90.543100,-90.803910,-103.409520,-83.588280,-83.588280,...,-82.371185,-103.844200,-72.460560,-75.329430,-125.925766,-81.501830,-82.197310,-82.197310,-116.362885,-89.934555
25%,1249.500000,2.000000,-9.345543,-9.258608,-9.519414,-9.693284,-9.432479,-9.693284,-10.127961,-10.562637,...,-7.780708,-7.259097,-7.432967,-7.432967,-6.998291,-6.911355,-6.814286,-6.824420,-6.780952,-6.389744
50%,2499.000000,2.000000,-0.391209,-0.652015,-0.652015,-0.738950,-0.738950,-0.999756,-1.260562,-1.347497,...,0.867644,1.086691,1.347497,1.434432,1.521367,1.869109,2.042979,2.129914,2.042979,2.477656
75%,3748.500000,3.000000,8.062149,8.041514,8.215385,8.215385,7.883883,7.780708,7.443468,7.172161,...,9.780220,10.301831,10.214896,10.301831,10.997314,10.823443,11.084250,11.301588,11.084250,11.258120
max,4998.000000,4.000000,105.409035,95.585350,91.934070,109.668860,87.065690,87.413430,77.850550,112.972404,...,97.758730,102.714040,93.498900,90.108420,87.326490,97.063250,91.586330,90.282295,84.544560,84.805374


In [6]:
eeg_df = data.drop(columns=['Unnamed: 0', '0'])
labels = data['0']

In [17]:
eeg_df.head()

,1,2,3,4,5,6,7,8,9,10,...,169,170,171,172,173,174,175,176,177,178
0,-1.608303,-13.170696,-8.997802,1.347497,5.259585,-1.521367,2.651526,4.651038,3.781685,-1.521367,...,-8.041514,-6.563614,-4.998779,-5.346520,-6.389744,-11.171185,-2.390720,1.086691,6.911355,6.128938
1,-2.825397,2.390720,3.955555,-4.216361,2.477656,2.129914,3.955555,-5.172649,5.694261,1.608303,...,4.129426,7.085226,-2.390720,-2.303785,-3.694750,4.564103,-6.215873,-2.738462,-4.477167,-4.390232
2,-15.952625,-3.347008,-15.517949,-9.432479,-16.474237,-6.650549,-14.909402,-12.214408,-10.823443,-7.259097,...,-4.390232,-0.738950,4.477167,12.996825,21.951160,21.081806,21.603418,14.909402,9.345543,6.998291
3,-26.558731,-18.560684,-12.040537,-4.216361,-9.432479,1.260562,-2.216850,-3.520879,-5.781197,-6.128938,...,-3.086203,-0.652015,3.607814,15.170208,16.039560,6.128938,3.955555,-1.956044,-2.390720,-4.651038
4,-5.607326,6.042003,-4.129426,-5.172649,-7.172161,-11.953602,-7.780708,-0.130403,-2.912332,-4.390232,...,-1.260562,1.608303,4.564103,-4.911844,10.301831,10.127961,8.389256,-2.216850,-4.390232,0.478144


In [18]:

# Step 2: Extract features
spectral_features_df = extract_spectral_features(eeg_df, sf=250)

# Step 3: Merge with labels and proceed to model training
# labels = pd.read_csv("labels.csv")
full_df = spectral_features_df.join(labels)


c:\Users\Bilal\anaconda3\envs\DM_env\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 178, using nperseg = 178
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
C:\Users\Bilal\AppData\Local\Temp\ipykernel_7052\2881372007.py:5: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(psd[idx_band], freqs[idx_band])  # Integral over the band


In [19]:
full_df.head()

,delta_power,delta_rel_power,theta_power,theta_rel_power,alpha_power,alpha_rel_power,beta_power,beta_rel_power,gamma_power,gamma_rel_power,0
0,0.694809,0.007408,3.528855,0.037625,2.384779,0.025427,52.798660,0.562941,34.787260,0.370903,0
1,0.882410,0.013944,13.706102,0.216590,0.759169,0.011997,26.704889,0.422003,19.731965,0.311814,0
2,3.962445,0.061655,6.003682,0.093417,0.407503,0.006341,38.606461,0.600714,9.194844,0.143071,0
3,2.601810,0.051251,6.318639,0.124466,1.973491,0.038874,18.368585,0.361829,20.410101,0.402043,0
4,2.977238,0.076321,2.601209,0.066682,0.289403,0.007419,11.521421,0.295351,22.100477,0.566545,0


In [8]:
X = eeg_df
# X_processed = spectral_features_df

y = labels

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.38      0.03      0.05       102
           1       0.50      0.07      0.12       140
           2       0.51      0.83      0.63       400
           3       0.70      0.79      0.74       208
           4       0.41      0.24      0.30       150

    accuracy                           0.54      1000
   macro avg       0.50      0.39      0.37      1000
weighted avg       0.52      0.54      0.48      1000



In [23]:
processed_X_train, processed_X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42, stratify=y)


In [24]:


clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(processed_X_train, y_train)

# Predict
y_pred = clf.predict(processed_X_test)

# Evaluate
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.58      0.50      0.54       102
           1       0.47      0.28      0.35       140
           2       0.56      0.65      0.60       400
           3       0.69      0.77      0.73       208
           4       0.46      0.40      0.43       150

    accuracy                           0.57      1000
   macro avg       0.55      0.52      0.53      1000
weighted avg       0.56      0.57      0.56      1000



we notice improvement in the accuracy, especially for the non-dominant class


# Wavelet transformation


In [11]:
# y = df.iloc[:, 0].values  # class labels
X = eeg_df.iloc[:, 1:].values  # time series signals

In [12]:
import pywt

# Choose wavelet and level
wavelet = 'db4'
level = 1

# Apply DWT to each row
X_wavelet = []

for row in X:
    coeffs = pywt.wavedec(row, wavelet, level=level)
    # coeffs[0] = approximation, coeffs[1:] = detail coefficients
    X_wavelet.append(coeffs[0])  # keep only approximation (or combine if you want more detail)

X_wavelet = np.array(X_wavelet)


In [13]:
print("Original shape:", X.shape)
print("Transformed shape:", X_wavelet.shape)


Original shape: (4999, 177)
Transformed shape: (4999, 92)


In [18]:
X_Wv_df = pd.DataFrame(X_wavelet)

In [19]:
from sklearn.discriminant_analysis import StandardScaler


scaler = StandardScaler()
X_Wv_df = scaler.fit_transform(X_Wv_df)

In [21]:
X_Wv_df = pd.DataFrame(X_Wv_df)


In [22]:
X_Wv_df.to_csv('../data/X_Wv_df.csv', index=False)